Create the training dataset

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)


# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


In [2]:
#!pip install 'numexpr==2.7'

In [3]:
np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

In [4]:
pd.DataFrame(X).describe()

,0,1,2
count,60.000000,60.000000,60.000000
mean,0.024067,0.209325,0.071554
std,0.835541,0.424468,0.213363
min,-1.157905,-0.643841,-0.476101
25%,-0.937910,-0.162549,-0.072426
50%,0.023472,0.300523,0.099102
75%,0.797043,0.601043,0.247797
max,1.174869,0.756048,0.467833


In [5]:
X_centered = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X_centered)
c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]

m, n = X.shape

S = np.zeros(X_centered.shape)
S[:n, :n] = np.diag(s)

W2 = Vt.T[:, :2]
X2D = X_centered.dot(W2)

X2D_using_svd = X2D

In [6]:
pd.DataFrame(X2D_using_svd).describe()

,0,1
count,6.000000e+01,6.000000e+01
mean,-3.885781e-17,2.775558e-18
std,8.822187e-01,3.676583e-01
min,-1.342194e+00,-6.970757e-01
25%,-9.578107e-01,-3.476975e-01
50%,1.376839e-01,1.465360e-01
75%,8.126351e-01,3.237334e-01
max,1.128726e+00,4.903635e-01


# Using Scikit-Learn


With Scikit-Learn, PCA is really trivial. It even takes care of mean centering for you:

In [7]:
from sklearn.decomposition import PCA

In [8]:
pca=PCA(n_components=2)
X2D=pca.fit_transform(X)

Notice how the results produced by two different approaches are almost the same except for the flipped axes

In [9]:
X2D[:5]

array([[ 1.26203346,  0.42067648],
       [-0.08001485, -0.35272239],
       [ 1.17545763,  0.36085729],
       [ 0.89305601, -0.30862856],
       [ 0.73016287, -0.25404049]])

In [10]:
X2D_using_svd[:5]

array([[-1.26203346, -0.42067648],
       [ 0.08001485,  0.35272239],
       [-1.17545763, -0.36085729],
       [-0.89305601,  0.30862856],
       [-0.73016287,  0.25404049]])

Recover the 3D points projected on the plane (PCA 2D subspace).

In [11]:
X3D_inv=pca.inverse_transform(X2D)

Note the values are almost the same except for some minor variances

In [12]:

X3D_inv[:5], X[:5]

(array([[-1.01450604, -0.54656333, -0.27441525],
        [-0.02103231,  0.55108376,  0.18101894],
        [-0.95379477, -0.4668077 , -0.24237013],
        [-0.91717404,  0.22083765, -0.01049779],
        [-0.74607229,  0.22027492,  0.00492637]]),
 array([[-1.01570027, -0.55091331, -0.26132626],
        [-0.00771675,  0.59958572,  0.03507755],
        [-0.95317135, -0.46453691, -0.24920288],
        [-0.92012304,  0.21009593,  0.02182381],
        [-0.76309739,  0.158261  ,  0.19152496]]))

PCA Components 

In [13]:
pca.components_

array([[-0.93636116, -0.29854881, -0.18465208],
       [ 0.34027485, -0.90119108, -0.2684542 ]])

In [14]:
Vt[:2]

array([[ 0.93636116,  0.29854881,  0.18465208],
       [-0.34027485,  0.90119108,  0.2684542 ]])

In [15]:
Vt.shape

(3, 3)

In [16]:
Vt

array([[ 0.93636116,  0.29854881,  0.18465208],
       [-0.34027485,  0.90119108,  0.2684542 ],
       [-0.08626012, -0.31420255,  0.94542898]])

Choosing the Right Number of Dimensions

In [17]:
pca=PCA()
pca.fit(X)

PCA()

In [18]:
pca.explained_variance_ratio_

array([0.84248607, 0.14631839, 0.01119554])

Find out how much variance has been covered in existing conponents

In [19]:
1-pca.explained_variance_ratio_.sum()

1.1102230246251565e-16

# Choosing the right number of dimensions

In [20]:
from sklearn.datasets import fetch_openml

mnist=fetch_openml('mnist_784', version=1, as_frame=False)
mnist.target=mnist.target.astype(np.uint8)

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X=mnist["data"]
y=mnist["target"]

In [23]:
X_train, x_test, y_train, y_test=train_test_split(X, y)

In [24]:
#help(train_test_split)

In [25]:
pca=PCA()
pca.fit(X_train)
cumsum=np.cumsum(pca.explained_variance_ratio_)

In [26]:
np.argmax(cumsum>=0.95)+1

154

Alternatively we can set the n_components between 0.1 and 1 to choose number of components

 pca = PCA(n_components=0.95)
 X_reduced = pca.fit_transform(X_train)

In [27]:
pca = PCA(n_components=0.75)
X_reduced = pca.fit_transform(X_train)

In [28]:
X_reduced.shape[1]

33

Array for features vs explained variances

In [29]:
pca.components_.shape, X.shape

((33, 784), (70000, 784))

Random PCA

In [30]:
import time

In [31]:
start=time.time()

rnd_pca = PCA(n_components=154, svd_solver="randomized")
X_reduced = rnd_pca.fit_transform(X_train)

end=time.time()
print(end-start)

4.728097915649414


In [32]:
start=time.time()
rnd_pca=PCA(n_components=154)
X_reduced = rnd_pca.fit_transform(X_train)
end=time.time()
print(end-start)

3.824439764022827


In [33]:
from sklearn.decomposition import IncrementalPCA
n_batches=100

inc_PCA=IncrementalPCA(n_components=154)

for X_batch in np.array_split(X_train, n_batches):
            inc_PCA.partial_fit(X_batch)
X_reduced=inc_PCA.transform(X_train)

In [34]:
X_train.shape, X_reduced.shape

((52500, 784), (52500, 154))

Kernel PCA

In [35]:
from sklearn.decomposition import KernelPCA

In [36]:
rbf_pca=KernelPCA(n_components=2, kernel='rbf', gamma=0.04)

How to select a kernel and tunning hyperparameters

In [37]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [ ]:
clf=Pipeline(
[("kpca", KernelPCA(n_components=2)), 
 ("log_reg", LogisticRegression())    
])

param_grid=[{"kpca__gamma": np.linspace(0.03,  0.05, 10),
             "kpca__kernel": ["rbf", "sigmoid"]
            }
    
]

In [ ]:
grid_search=GridSearchCV(clf, param_grid, cv=3, verbose=10)
grid_search.fit(X, y)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV 1/3; 1/20] START kpca__gamma=0.03, kpca__kernel=rbf.........................
